In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
Nghost = 2 # Number of ghost zones

# Define a class to hold the parameters of the hydrodynamics simulation
# and run the sim and handled the results
class Burgers(object):
    def __init__(self, Ncells, IC, outCad=0): 
        self._outCad = outCad
        self._t = 0.0
        self._N = Ncells+2*Nghost
        self._dx = 1.0/Ncells 
        self._x = np.linspace((0.5-Nghost)*self._dx,1+(Nghost-0.5)*self._dx, Ncells+2*Nghost)
        self._u = self._getIC(IC, self._x) 
        self._dt = 0.25*self._dx
    def _getIC(self,IC, x):
        if (IC=="sine"):
            return 0.35+0.25*np.sin(4.0*np.pi*x)
        elif (IC=="step"):
            f = np.ones_like(x)
            f[abs(x-0.5)<0.25] = 0.5
            return f
        else:
            raise ValueError("IC undefined") 
    def _physFlux(self, u):
        return 0.5*u**2
    def _flux(self, ul, ur): #ul/ur is left/right of interface
        #Roe's method
        uavg = 0.5*(ul+ur)
        delta = ur-ul
        return (
                0.5*(self._physFlux(ul)+self._physFlux(ur))
                -0.5*abs(uavg)*delta
               )
    def _uPlusMinus(self, u):
        #Min-mod
        slope = np.zeros_like(u)
        deltau = u[1:]-u[:-1]
        slope[1:-1] = deltau[1:]
        I = abs(deltau[1:]) > abs(deltau[:-1])
        slope[1:-1][I] = deltau[:-1][I]
        slope[1:-1][deltau[1:]*deltau[:-1]<=0] = 0
        return (u+0.5*slope,u-0.5*slope)
    def _udot(self, u):
        #Calculate left and right hand states at interfaces
        up,um = self._uPlusMinus(u)
        F = np.zeros(len(u)+1)
        #Calculate flux at interfaces
        F[1:-1] = self._flux(up[:-1],um[1:])
        return (1.0/self._dx)*(F[:-1]-F[1:])
    def _applyBC(self, u):
        #Copy over ghost cells assuming periodic BCs
        ubc = 1.0*u
        ubc[0:Nghost] = u[-(2*Nghost):-Nghost]
        ubc[-Nghost:] = u[Nghost:2*Nghost]
        return ubc
    def _step(self):
        #Use Runge-Kutta 2
        uhalf = self._u + 0.5*self._dt*self._udot(self._u)
        uhalf = self._applyBC(uhalf)
        self._u += self._dt*self._udot(uhalf)
        self._u = self._applyBC(self._u)
        self._t += self._dt
    def evolve(self, tf, save=True, show=False):
        while(self._t<tf):
            if (self._outCad>0): self._output(save=save, show=show)
            self._step()
        if (self._outCad>0): self._output(save=save, show=show)
    def _output(self, save=True, show=False):
        nstep = np.rint(self._t/self._dt)
        if (nstep%self._outCad!=0): return
        plt.clf()
        plt.ylabel(r"u")
        plt.xlabel(r"x")
        plt.title("t=%1.2f"%self._t)
        plt.plot(self._x[Nghost:-Nghost], self._u[Nghost:-Nghost], "k-")
        #plt.ylim([0.1,0.6])
        if show:
            plt.show()
        if save:
            plt.savefig("fig_%d.png" % (nstep/self._outCad)) 
    def plotReconstruction(self):
        up,um = self._uPlusMinus(self._u)
        stencil = 0.5*self._dx*np.array([-1,0,1])
        plt.plot(self._x[Nghost:-Nghost],self._u[Nghost:-Nghost],"ro")
        for n in range(Nghost,self._N-Nghost):
            plt.plot(stencil+self._x[n],[um[n],self._u[n],up[n]],"k")
        plt.xlabel(r"$x$")
        plt.ylabel(r"$u$")
        plt.show()
    def getSoln(self):
        return self._u[Nghost:-Nghost]
    def getDomain(self):
        return self._x[Nghost:-Nghost]
    def getMass(self):
        return self._dx*np.sum(self._u[Nghost:-Nghost])


In [ ]:
# delete output files from a possible previous run
os.system('rm fig*.png animation.mp4')

# Parameter values:

# Number of cells in the simulation
Ncells = 64

# Initial conditions -- 'sine' or 'step'
IC = 'sine'

# Output cadence -- how often to produce a plot
outCad = 8

# Final time -- how long to run the simulation for
tf = 1.0

burgers = Burgers(Ncells, IC, outCad=outCad)
# If you pass 'show=True', it will display a series of plots;
# if you pass 'save=True' (the default), it will also save a series of .png plots...
burgers.evolve(tf, show=True)

# ... which you can convert into a movie "animation.mp4".
# If you're running on Symmetry, you will have to download the animation.mp4 file to your computer to view it.
os.system('ffmpeg -r 12 -i fig_%d.png -qscale:v 0 -pix_fmt yuv420p -c:v libx264 -crf 20 -r 24 animation.mp4');

In [ ]:
def lerp(xs, ys, x):
    '''
    Evaluate the linear interpolation of the given sample points at the given point.
    Constant extrapolation is performed beyond the endpoints.
    
    Assume xs is sorted (in increasing order).
    List lengths must be equal.
    Lists must be nonempty.
    '''
    if len(xs) != len(ys):
        raise ValueError('Length mismatch.')
    n = len(xs)
    if n == 0:
        raise ValueError('Empty sample lists.')
    i = np.searchsorted(xs, x)
    if i == 0:
        return ys[0]
    elif i >= len(xs):
        return ys[-1]
    elif x == xs[i]:
        return ys[i]
    else:
        # xs[i - 1] < x < xs[i]
        dx = xs[i] - xs[i - 1]
        dy = ys[i] - ys[i - 1]
        return dy / dx * (x - xs[i - 1]) + ys[i - 1]

def resample(ys, n2):
    '''
    Sample the linear interpolation of the given ordinates on a uniformly sampled range
    to a new uniformly sampled range.
    '''
    n = len(ys)
    xs = np.linspace(0.0, 1.0, n)
    return np.array([lerp(xs, ys, x) for x in np.linspace(0.0, 1.0, n2)])

def quad(h, ys):
    '''
    Return area under linearly interpolated ordinates with abscissas separated
    by a distance of h.
    '''
    return h*(np.sum(ys) - .5*ys[0] - .5*ys[-1])

def norm(h, ys):
    '''
    Return L2 norm of linearly interpolated ordinates with abscissas separated
    by a distance of h.
    '''
    return np.sqrt(quad(h, ys**2))

def dist(h, us, vs):
    '''
    Return L2 distance of linearly interpolated ordinates with abscissas
    separated by a distance of h.
    '''
    return norm(h, us - vs)

def log2space(start, num):
    '''
    Return all doublings specified number of times.
    '''
    if num < 0:
        raise ValueError('Negative number.')
    elif num == 0:
        return []
    else:
        return [start] + log2space(2*start, num - 1)

def adjacent(f, xs):
    '''
    Apply f to adjacent pairs of the given list.
    '''
    return [f(xs[i], xs[i + 1]) for i in range(0, len(xs) - 1)]

In [ ]:
'''
Method 1
'''
def order(t, s0, num):
    '''
    Estimate the convergence power law of Burger's equation solver using the
    distance between three consecutive solutions at doubling resolutions. Do
    this multiple times and take the average of the parameters.

    This method approximates the distance between two consecutive solutions
    as the sum of their (true) errors. This amounts to approximating the
    triangle inequality as equality.
    
    t   : Evolution time
    s0  : Number of samples of first simulation
    num : Number of simulations to run, each with double the previous number
        of samples
    ->  : (As, ns) where
        As are the estimates of the proportionality coefficient A
        ns are the estimates of the convergence order n where
            error = A*resolution^n
        Note len(As) = len(ns) = num - 2
    '''
    samps = log2space(s0, num)
    N = num - 2
    As = np.zeros(N)
    ns = np.zeros(N)
    for i in range(N):
        s1 = samps[i]
        s2 = samps[i + 1]
        s3 = samps[i + 2]
        # initialize
        f1 = Burgers(s1, 'sine')
        f2 = Burgers(s2, 'sine')
        f3 = Burgers(s3, 'sine')
        # evolve
        f1.evolve(t)
        f2.evolve(t)
        f3.evolve(t)
        # upsample
        s = s3
        h = 1./s
        u1 = resample(f1.getSoln(), s)
        u2 = resample(f2.getSoln(), s)
        u3 = resample(f3.getSoln(), s)
        # adjacent distances
        d1 = dist(h, u1, u2)
        d2 = dist(h, u2, u3)
        # power law estimate
        ns[i] = np.log2(d1/d2)
        As[i] = d1/(h**ns[i]*(1 - 2**(-ns[i])))
    return (As, ns)

def plotOrder(t, s0, num):
    (As, ns) = order(t, s0, num)
    n = np.mean(ns)
    plt.plot(ns, 'o', [0, len(ns) - 1], [n, n])
    plt.xlabel('Index')
    plt.ylabel('Convergence Order Estimate')
    plt.title('Method 1, t = {:.3g}'.format(t))
    plt.legend(['Simulations', 'Mean = {:.3g}'.format(n)])
    if t < 0.3:
        # before shocks
        plt.savefig("bs1.png")
    else:
        # after shocks
        plt.savefig("as1.png")

def plotEvolution(t1, t2, numT, s1, numS):
    ts = np.linspace(t1, t2, numT)
    ns = [np.mean(order(t, s1, numS)[1]) for t in ts]
    plt.plot(ts, ns, 'o')
    plt.xlabel('Time')
    plt.ylabel('Convergence Order Estimate')
    plt.title('Method 1 Evolution')
    plt.savefig("evol1.png")

In [ ]:
'''
Method 2
'''
def fitPow(xs, ys):
    '''
    Fit a power law to the given data.
    
    Return (c, e) such that ys = A*xs^e with least squares error.
    '''
    A = np.array([[1, np.log(x)] for x in xs])
    b = np.log(ys)
    x = np.linalg.lstsq(A, b, rcond=None)
    return (np.exp(x[0][0]), x[0][1])

def order2(t, s0, num):
    '''
    Estimate the convergence power law of this Burger's equation solver by
    fitting a power law to the approximate errors of all simulations at
    doubling resolutions.
    
    This method approximates the true error of each solution as the distance
    to the solution with the highest resolution.
    
    t   : Evolution time
    s0  : Number of samples of first simulation
    num : Number of simulations to run, each with double the previous number
        of samples
    ->  : (hs, es, A, n) where
        hs are the resolutions used for each solution
        es are the approximate errors of each solution
        A  is the estimate of the proportionality coefficient
        n  is the estimate of the convergence order where
            error = A*resolution^n
        Note len(As) = len(ns) = num - 1
    '''
    samps = log2space(s0, num)
    fs = [Burgers(s, 'sine') for s in samps]
    for f in fs:
        f.evolve(t)
    hs = 1./np.array(samps)
    s = samps[-1]
    h = 1./s
    us = [resample(f.getSoln(), s) for f in fs]
    es = [norm(h, u - us[-1]) for u in us[:-1]]
    A, n = fitPow(hs[:-1], es)
    return (hs[:-1], es, A, n)

def plotOrder2(t, s0, num):
    hs, es, A, n = order2(t, s0, num)
    xplot = np.linspace(np.min(hs), np.max(hs), 1000)
    # plt.plot(hs, es, 'o', xplot, A*xplot**n)
    plt.loglog(hs, es, 'o', xplot, A*xplot**n)
    plt.xlabel('Resolution')
    plt.ylabel('Error Estimate')
    plt.title('Method 2, t = {:.3g}'.format(t))
    plt.legend(['Simulations', 'y = {:.3g}*x^{:.3g}'.format(A, n)])
    if t < 0.3:
        # before shocks
        plt.savefig("bs2.png")
    else:
        # after shocks
        plt.savefig("as2.png")

def plotEvolution2(t1, t2, numT, s1, numS):
    ts = np.linspace(t1, t2, numT)
    ns = [order2(t, s1, numS)[3] for t in ts]
    plt.plot(ts, ns, 'o')
    plt.xlabel('Time')
    plt.ylabel('Convergence Order Estimates')
    plt.title('Method 2 Evolution')
    plt.savefig("evol2.png")

## Convergence Order Estimate (Before Shocks)

In [ ]:
plotOrder(0.2, 64, 6)

In [ ]:
plotOrder2(0.2, 64, 6)

## Convergence Order Estimate (After Shocks)

In [ ]:
plotOrder(1.0, 64, 6)

In [ ]:
plotOrder2(1.0, 64, 6)

## Convergence Order Evolution

In [ ]:
plotEvolution(0.0, 1.0, 20, 64, 6)

In [ ]:
plotEvolution2(0.0, 1.0, 20, 64, 6)